<a href="https://colab.research.google.com/github/WellingtonRmos/MLearning/blob/main/HMMLearn%5BPT_BR%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tutorial de HMMLearn

Data de atualização: 04 de julho de 2023

Video sobre o modelo [EN]: (https://www.youtube.com/watch?v=RWkHJnFj5rY) autor NormalizedNerd

# 1.0 Objetivo do tutorial

Tudo bem, o objetivo do tutorial é tirar alguém do zero é ser capaz de criar seu primeiro modelo HMM com python. Não iremos explorar todas as possibilidades aqui, mas já terá uma visão de como procurar é organizar seu codígo futuro

Vamos aprender:

* Importar as principais bibliotecas
* Usar o yfinance para obter os dados
* Dividir entre treino e teste para series temporais
* Como usar os states para observar e dividir o dataset

Não vamos criar um modelo lucrativo o intuito não é esse, porém ter fundamentos e bases solidas sobre o modelo para trabalhos futuros

# 1.1 Instalando o yfinance e hmmlearn
Basta instalar usando pip como qualquer outro pacote

In [ ]:
!pip install yfinance
!pip install hmmlearn

# 2. Importar as bibliotecas é preencher o DataFrame
Aqui é uma rotina padrão. O unico passo diferente é a forma como os retornos são calculados. O comando '.pct_change' poderia ser usado sozinho também geraria retornos, MAS é recomendado utilizar Log returns.

In [ ]:
import pandas as pd
import numpy as np
from yfinance import download
from hmmlearn.hmm import GaussianHMM
pd.options.mode.chained_assignment = None  # default='warn'

# Selecione um periodo, intervalo e o nome da acao
time = '1wk'
start_date = '2010-01-01'
stock = 'nvda'

# Criar e preencher o DataFrame
df = download(stock, interval=time, start=start_date)[['Open', 'High', 'Low', 'Close']]

# Criar novas colunas para o log return e o retorno total acumulado
df['Ret'] = np.log(df.Close.pct_change() + 1)
df['Hold'] = np.exp(df.Ret.cumsum())

df.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Visualizar o DataFrame
df.head(11)

# 3. Dividir o DataFrame
1. Primeiro Selecionamos o tamanho do corte
2. Criamos um novo DataFrame para armazenar apenas os dados que queremos
3. Limpamos o DataFrame de possíveis Na
4. Dividimos entre treino e teste e cortamos os dados onde selecionamos

In [ ]:
# Criar um corte nos dados
tamanho = .25
t = round( len(df) * tamanho)

# Criar o DataFrame e as colunas para o modelo
dff = pd.DataFrame()
dff['Ret'] = df.Ret
dff['Labels'] = pd.qcut(df.Ret, q=4, labels=[0, 1, 2, 3]) # Divide os dados em quatro categorias
dff['BGER_MEAN'] = np.where(df.Ret > df.Ret.mean(), 1, 0) # Seleciona onde os dados sao maiores que a media

# Remover Na
dff.dropna(inplace=True)

# Separar os dados entre treino e teste
X_train = dff[["Ret", "Labels", "BGER_MEAN"]].iloc[:t]
X_test = dff[["Ret", "Labels", "BGER_MEAN"]].iloc[t:]

In [ ]:
# Visualizar o novo DataFrame de teste
X_train

# 4. Criar o modelo
Aqui o processo é simples primeiro criamos o modelo é depois fazemos as predições nos dados de test é armazenamos em um novo DataFrame deletar a os dados de teste é completamente opcional

In [ ]:
# Processar os dados no modelo HMM
model = GaussianHMM(n_components=4, covariance_type="diag", n_iter=1000, random_state=7).fit(X_train)
hidden_states = model.predict(X_test)

# Criar um novo DataFrame mais legivel
df1 = X_test.copy()
del X_test

# 5. Analisar os estados e resultados
Aqui é  o que importa de fato, como os dados de acordo com o HMM estão separados e ordenados
1. Criamos uma nova coluna para os estados
2. Selecionamos onde os estados forem == 1, 2, por exemplo, é adicionamos 1 para calculos futuros.
    * O passo 2 é o mais importante de ser intendido, pois é onde os dados são separados é devem vir a ser escolhido de acordo com a necessidade do modelo
3. Calcula-se o retorno do dia futuro * o sinal escolhido do dia presente. Resultando no retorno da estratégia nesse caso.
4. Calculo do retorno acumulado da ação
5. Retorno a juros simples da estratégia

In [ ]:
# Preencher as colunas do novo DataFrame, Selecionar os estados de desejo, calcular o retorno, B&H, Soma dos juros simples da estrategia, Plots
df1['State'] = hidden_states
df1['Sinal'] = np.where((df1.State == 4) | (df1.State == 1), 1, 0)
df1['STRATEGY_RETURN'] = df1.Sinal * (df1.Ret.shift(-1))
df1['Hold'] = np.exp((df1.Ret).cumsum())
df1['STRATEGY_SIMPLE_RETURNS'] =  ((df1.STRATEGY_RETURN ).cumsum())
df1.STRATEGY_SIMPLE_RETURNS.plot()

# 5.1 Mais explicações

Note que o que faz o resultado ser diferente está nessa linha de codigo é você deve entender isso claramente


In [ ]:
# Mudaremos os estados para 0 e 1 por exemplo
df1['Sinal'] = np.where((df1.State == 0) | (df1.State == 1), 1, 0)

# Copia do resto do codigo
df1['STRATEGY_RETURN'] = df1.Sinal * (df1.Ret.shift(-1))
df1['Hold'] = np.exp((df1.Ret).cumsum())
df1['STRATEGY_SIMPLE_RETURNS'] =  ((df1.STRATEGY_RETURN ).cumsum())
df1.STRATEGY_SIMPLE_RETURNS.plot()

# 5.2 Dividir o dataset

Você também pode dividir o dataset pelas suas propriedades estatisticas

In [ ]:
# Exemplo de como selecionar os dados por 'state'
estado_1 = df1.loc[df1.State == 2]
estado_1.drop(columns=['STRATEGY_SIMPLE_RETURNS', 'STRATEGY_RETURN'], inplace=True)
estado_1

,Ret,Labels,BGER_MEAN,State,Sinal,Hold
Date,,,,,,
2013-05-31,-0.026317,1,0,2,0,0.974026
2013-06-21,-0.030231,0,0,2,0,0.957621
2013-11-01,-0.043046,0,0,2,0,0.994532
2013-11-15,-0.056433,0,0,2,0,1.047847
2013-12-06,-0.038304,0,0,2,0,1.032809
...,...,...,...,...,...,...
2022-12-16,-0.099987,0,0,2,0,41.938482
2022-12-23,-0.049172,0,0,2,0,39.926178
2023-06-02,-0.032195,0,0,2,0,105.290500


# Conclusão
Espero que tenha aprendido um pouco sobre como usar o HMM e tenha agora algum conhecimento básico da ferramenta nossos resultados nesse teste não foram relevantes, porém a ideia não era criar um modelo lucrativo, mas sim passar o conhecimento adiante. Caso tenha sido util o objetivo foi atingido

# 6. Codigo como exemplo
exemplo de calculos simples para ações

In [ ]:
# Calculo indice de acerto
up = len(df.loc[df.Ret > 0]) / len(df)
down = 1 - up
1 - down

# Media de retornos positivos e negativos
media_alta = df.Ret.loc[df.Ret > 0].mean()
media_baixa = df.Ret.loc[df.Ret < 0].mean()

# Medias dos retornos de alta e baixa
print(f"Probabilidade de alta  {up:.2f}%")
print(f"Probabilidade de baixa {down:.2f}%")
print(f"O retorno medio foi de nos dias de alta  {media_alta:.4f}%")
print(f"O retorno medio nos dias de baixa       {media_baixa:.4f}%")

# Kelly fraction //  Fomula para Calcular o criterio de kelly apenas por curiosidade
f = (abs(media_alta / media_baixa) * up - down) / abs(media_alta / media_baixa)
print(f"A fracao ideal de aporte é de {f:.2f}%")

Probabilidade de alta  0.56%
Probabilidade de baixa 0.44%
O retorno medio foi de nos dias de alta  0.0448%
O retorno medio nos dias de baixa       -0.0435%
A fracao ideal de aporte é de 0.14%
